In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

df = pd.read_csv('experiment1.0.csv')
id = ['NACCID', 'NACCVNUM']
cross_sectional = ['EDUC', 'COGFLAGO']
longitudinal = ['DECIN', 'DECCLIN', 'COGMEM', 'COGORI', 'COGJUDG', 'COGLANG', 'COGVIS', 'COGATTN', 'COGFLUC', 'COGOTHR', 'PROBAD']
visits = {}
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    naccid = row['NACCID']
    naccvnum = row['NACCVNUM']
    # If the id is not already in the dictionary, add it with the current visit number
    if naccid not in visits:
        visits[naccid] = naccvnum
    # If the id is already in the dictionary, update the visit number if the current one is larger
    else:
        visits[naccid] = max(visits[naccid], naccvnum)

vectors = {naccid: np.full(21, np.nan) for naccid in df['NACCID'].unique()}

# ...

def fill_vectors(variable):
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        naccid = row['NACCID']
        naccvnum = int(row['NACCVNUM'])
        response = row[variable]
        educ = row['EDUC']  # get the 'educ' value

        # If naccvnum is 1, insert the 'educ' value at the first index
        if naccvnum == 1:
            vectors[naccid][0] = educ

        # Subtract 1 from naccvnum to get the correct index for the response
        vectors[naccid][naccvnum] = response

for variable in longitudinal:
    variable_name = variable
    fill_vectors(variable)
    df_vectors = pd.DataFrame.from_dict(vectors, orient='index')
    df_vectors.to_csv(f'{variable_name} vectors.csv', index=False)

average_visits = sum(visits.values()) / len(visits)
min_visits = min(visits.values())
max_visits = max(visits.values())
total_persons = len(visits)
# print(f'Average number of visits per person: {average_visits}')
# print(f'Minimum number of visits per person: {min_visits}')
# print(f'Maximum number of visits per person: {max_visits}')
# print(f'Total number of unique persons: {total_persons}')

100%|██████████| 185831/185831 [00:05<00:00, 36147.58it/s]


Ranking vectors according to 


In [23]:

import pandas as pd
import numpy as np

df = pd.read_csv('COGMEM vectors.csv', header=None)
df = df.dropna()
# Define a function to categorize vector rows
def categorize_memory_responses(row):
    # Check if all values are 0 or NaN
    if all(val == 0 or pd.isna(val) for val in row[1:]):
        return 'Only 0s or NaNs'
    
    # Check if all values are 1 or NaN
    elif all(val == 1 or pd.isna(val) for val in row[1:]):
        return 'Only 1s or NaNs'
    
    # Check if there is a mix of 1s, 0s, and NaNs
    elif any(val == 1 for val in row[1:]) and any(val == 0 for val in row[1:]):
        return 'Mix of 1s and 0s'
    
    # Check if all values are NaN
    elif all(pd.isna(val) for val in row[1:]):
        return 'Only NaNs'
    
    # Default case
    else:
        return 'Other'

# Bin the 'educ' variable into groups of 4
df['educ_bins'] = pd.cut(df[0], bins=np.arange(0, 40, 4), right=False)

# Apply the categorization function to each row
df['Category'] = df.apply(categorize_memory_responses, axis=1)

# Group by 'educ_bins' and count occurrences of each category
grouped_df = df.groupby(['educ_bins', 'Category']).size().unstack().fillna(0)

# Calculate the percentage of each category within each bin
grouped_df_percentage = grouped_df.div(grouped_df.sum(axis=1), axis=0) * 100

# Display the resulting DataFrame
print(grouped_df_percentage)


Category   Other
educ_bins       
[0, 4)     100.0
[4, 8)       NaN
[8, 12)      NaN
[12, 16)     NaN
[16, 20)     NaN
[20, 24)     NaN
[24, 28)     NaN
[28, 32)     NaN
[32, 36)     NaN


C:\Users\Andrew\AppData\Local\Temp\ipykernel_3260\1987503150.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = df.groupby(['educ_bins', 'Category']).size().unstack().fillna(0)
